---

# 🚥SECTION4 프로젝트 주제 - 한국도로에 맞는 신호등 인식 모델 만들기
---
<br><br>

#💡프로젝트를 진행하는 만드는 이유  
SECTION2 프로젝트를 자율주행을 주제로 진행하면서 알게 된 사실이 있습니다.

Section2 project 당시 데이터는 
2020년 12월 1일과 2021년 11월 30일 사이 <br>
캘리포니아 DMV(= 차량관리국 )에
캘리포니아 공공도로에서 자율주행차량을 테스트할 수 있는 허가를 받은 각 회사는<br>
DMV(= 차량관리국)에 자율주행시스템을 해제하고 운전한 이유를 반드시 보고해야 합니다.<br><br>
자율주행시스템의 해제를<br>
"자율주행시스템이 오류가 감지되거나
차량의 안전한 작동을 위해 자율주행차량 테스트운전자가 자율주행모드를 해제하고<br>
즉각적인 수동운전을 취해야 할 때 자율주행 모드가 비활성화되는 것"으로 정의합니다.<br><br>



### ✏️ 가장 많은 해제 이유 중 하나가 정지신호, 끼어드는 차량 등 공통적으로 차량이 인식을 늦게 해서 안전을 위해 사람이 개입했다고 많은 제조사들은 캘리포니아 자동차관리국에 신고하였습니다.
<br><br>

---

# 🔔그래서 이번 SECTION4 PROJECT는<br> 한국전자통신연구원에서 공개한 신호등데이터셋을 통해<br> 한국도로에 맞는 신호등인식모델을 만들어 보고,<br> <br>**신호등을 늦게 인식하는 문제를 인식모델로만 성능을 높여서 해결할 문제인지** 아니면<br> 다른 해결방법을 찾아야 하는지 알고 싶어서 진행하게 되었습니다.<br> 


---

🔆 데이터 출처 - [한국전자통신연구원 ETRI AI 나눔 - 신호등 데이터셋](https://nanum.etri.re.kr/share/kimjy/etri_traffic_light?lang=ko_KR) <br> <br> 

한국전자통신연구원에서 제공한 10000장의 이미지 데이터 중 데이터셋의 크기가 너무 커서 <br> 

✅그중 일부인 **2019년04월17일에 서울도로에서 테스트 주행 중 얻은
1900장의 이미지데이터로 프로젝트를 진행**하였습니다.<br> 


---



🔔 YOLO 설치하기

In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 13441, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 13441 (delta 177), reused 226 (delta 167), pack-reused 13198
Receiving objects: 100% (13441/13441), 13.24 MiB | 9.15 MiB/s, done.
Resolving deltas: 100% (9241/9241), done.


In [2]:
# 설치된 YOLO 5폴더 안 파일리스트 확인
!ls yolov5

benchmarks.py	 detect.py   models	       setup.cfg       val.py
classify	 export.py   README.md	       train.py
CONTRIBUTING.md  hubconf.py  requirements.txt  tutorial.ipynb
data		 LICENSE     segment	       utils


In [3]:
# yolo5을 쓰기 위해 requirements.txt로  환경을 맞쳐주기 위해 설치 함
!pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 41.3 MB/s 


In [4]:

#glob는 파일들의 리스트를 뽑을 때 사용
from glob import glob

img_list = glob('/content/drive/MyDrive/Section4_project/dataset/20190417_01_Seoul/images/*.jpg')

# dataset폴더안에 가지고 있는 이미지가 몇장인지 출력해봄
print(len(img_list))


1900


In [5]:
#훈련세트와 검증세트로 나눠줌

from sklearn.model_selection import train_test_split
#내가 헷갈렸던 부분
#train은 80% val데이터 셋은 20% 나눔
# train_test_split의 파라미터에 test_size지정 했다고
# train 80% 중에 val(검증)세트를 따로 나눠서
# train,val,test 총3개로 나누는 게 아니라
# train_test_split함수는 2가지로만 나눌 수 있다.
train_img_list, val_img_list = train_test_split(img_list, test_size = 0.2 ,random_state = 1000)

#전체 1900장 중에서 나눠 진 train, val의 데이터셋 크기 확인
print(len(train_img_list), len(val_img_list))

1520 380


✅전체 데이터 셋 이미지 1900장이<br>
- train 데이터 셋은 1520장<br>
- val(검증) 데이터 셋은 380 장으로 나눠짐<br><br>

---

In [6]:
# train 데이터셋 이미지 경로랑 
# val(검증)데이터셋 이미지 경로를 .txt파일로 저장해줌

with open ('/content/drive/MyDrive/Section4_project/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n' )

with open ('/content/drive/MyDrive/Section4_project/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n' )


#어떤 파일들이 train과 val의 데이터셋이 됐는지 리스트로 있는 것을 
#.txt파일로 만들어 줌
#파일을 열면 해당 이미지들의  경로명이 나열되어 있음

In [7]:
# data.yaml이  뭘 포함하는지 보려고
# cat(concatenate) : 단순 파일 출력, 파일 여러개를 합치는 기능도 가능
%cat /content/drive/MyDrive/Section4_project/dataset/data.yaml

names:
- Traffic Light
nc: 1
train: /content/drive/MyDrive/Section4_project/dataset/train.txt
val: /content/drive/MyDrive/Section4_project/dataset/val.txt


In [8]:
# data.yaml의 내용을 'r'읽어보고

import yaml

with open('/content/drive/MyDrive/Section4_project/dataset/data.yaml', 'r') as f:
  data = yaml.safe_load(f)
# data = yaml.load(f)하니깐 에러났는데 safe_load(f)쓰니깐 해결남
# 해결책 참고 https://stackoverflow.com/questions/70981856/how-do-i-fix-the-typeerror-load-missing-1-required-positional-argument-load
print(data)

data['train'] = '/content/drive/MyDrive/Section4_project/dataset/train.txt'
data['val'] = '/content/drive/MyDrive/Section4_project/dataset/val.txt'
# 내용수정 하려고
with open('/content/drive/MyDrive/Section4_project/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['Traffic Light'], 'nc': 1, 'train': '/content/drive/MyDrive/Section4_project/dataset/train.txt', 'val': '/content/drive/MyDrive/Section4_project/dataset/val.txt'}
{'names': ['Traffic Light'], 'nc': 1, 'train': '/content/drive/MyDrive/Section4_project/dataset/train.txt', 'val': '/content/drive/MyDrive/Section4_project/dataset/val.txt'}


In [9]:
#현재 위치 확인
%pwd

'/content'

In [10]:
# yolo5 설치한곳으로 이동
%cd /content/yolov5/

# 모델을 훈련시키기 위해
# 이미지 크기는 416으로 하고
# chg는 모델의 구조를 나타내는데 위치는 
# /content/yolov5/models 에
# yolo5l.yaml, yolo5m.yaml, yolo5n.yaml, yolo5s.yaml, yolo5x.yaml 
# 파일들을 말한다.

# 그중에 /content/yolov5/models/yolov5s.yaml모델을 써본다다

!python train.py --img 416 --batch 16 --epochs 50 --data /content/drive/MyDrive/Section4_project/dataset/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name Traffic_Light_yolov5s_results


# --weights yolov5s.pt 는 가중치는 pre-train 된 모델을 다운로드 받아서 사용해라는 뜻
# --name Traffic_Light_yolov5s_results 실행결과 나온 결과의 이름을 지정함함

/content/yolov5
train: weights=yolov5s.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/drive/MyDrive/Section4_project/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Traffic_Light_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-186-g7f097dd Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box

In [ ]:
# 내가 하고 있는게 맞는지.. 왜 트레이닝은 안될까요.. 레이블폴더도 있는데...